In [4]:
import tweepy
import csv
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker
import pandas as pd
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker



auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth,wait_on_rate_limit=True)

In [5]:
Session = sessionmaker()
engine = create_engine('sqlite:///twitter.db')
Session.configure(bind=engine)

In [6]:
hashtags = ['h1b', 'visa']

In [ ]:
for hashtag in hashtags:
    with open(hashtag+'.csv', 'r') as csvfile, open(hashtag+'-retweets.csv', 'a') as csvRetweetsFile:
        csvRetweetsWriter = csv.writer(csvRetweetsFile)
        spamreader = csv.reader(csvfile, delimiter=',')
        next(spamreader) # skip header
        for tweet in spamreader:
            if(len(tweet)>0):
                print("Looking for retweets of", tweet[0]) # tweet id
                retweets = [status for status in api.retweets(id=tweet[0])]
                if len(retweets)>0:
                    print("Found", len(retweets))
                    for retweet in retweets:
                        print("Retweet: ", retweet.id)
                        csvRetweetsWriter.writerow([retweet.id, retweet.retweeted_status.id, retweet.text.encode("utf-8"),
                        retweet.created_at, retweet.user.id])

In [52]:
csvfile.close()
csvRetweetsFile.close()

In [54]:
for hashtag in hashtags:
    tweetData = pd.read_csv(hashtag+'.csv', usecols=['tweet_id', 'retweeted', 'user_id', 'tweet_text','tweet_created_at','tweet_retweet_count','tweet_favorite_count'])
    userData = pd.read_csv(hashtag+'.csv', usecols=['user_id','user_name','user_location','user_friends_count'])
    retweetData = pd.read_csv(hashtag+'-retweets.csv', usecols=['retweet_id','source_tweet_id','tweet_text','tweet_created_at', 'user_id'])

    tweetData.to_sql('tweet', engine, if_exists='append')
    userData.to_sql('user', engine, if_exists='append')
    retweetData.to_sql('retweet', engine, if_exists='append')

In [23]:
resAll = pd.read_sql_query('SELECT hashtag, COUNT(DISTINCT tweet_id) \
FROM tweet \
GROUP BY hashtag', engine)
print(resAll)

  hashtag  COUNT(DISTINCT tweet_id)
0     h1b                      1039
1    visa                      1703


In [7]:
resH1b = pd.read_sql_query('SELECT COUNT(DISTINCT tweet_id), DATE(tweet_created_at) \
FROM tweet \
WHERE tweet.hashtag = \'h1b\' \
GROUP BY DATE(tweet_created_at) ', engine)
print(resH1b)

   COUNT(DISTINCT tweet_id) DATE(tweet_created_at)
0                       183             2018-10-20
1                       176             2018-10-21
2                       253             2018-10-22
3                       418             2018-10-23
4                         9             2018-10-24


In [8]:
resVisa = pd.read_sql_query('SELECT COUNT(DISTINCT tweet_id), DATE(tweet_created_at) \
FROM tweet \
WHERE tweet.hashtag = \'visa\' \
GROUP BY DATE(tweet_created_at) ', engine)
print(resVisa)

   COUNT(DISTINCT tweet_id) DATE(tweet_created_at)
0                       266             2018-10-20
1                       146             2018-10-21
2                       445             2018-10-22
3                       456             2018-10-23
4                       390             2018-10-24


In [9]:
resUser = pd.read_sql_query('SELECT tweet.hashtag, COUNT(DISTINCT user.user_id) FROM user \
INNER JOIN tweet ON tweet.user_id = user.user_id \
GROUP BY tweet.hashtag', engine)
print(resUser)

  hashtag  COUNT(DISTINCT user.user_id)
0     h1b                           413
1    visa                          1254


In [13]:
resRetweet = pd.read_sql_query('SELECT tweet.hashtag, COUNT(DISTINCT retweet.retweet_id) FROM retweet \
INNER JOIN tweet ON tweet.tweet_id = retweet.source_tweet_id \
GROUP BY tweet.hashtag', engine),
print(resRetweet)

(  hashtag  COUNT(DISTINCT retweet.retweet_id)
0     h1b                                  64,)


In [16]:
# Korelacja pomiędzy liczbą retweetów posta a firends_count usera
# Wskazanie kiedy powstały posty, które najczęściej retweetowano

resRetweetUserFriends = pd.read_sql_query('SELECT tweet.tweet_id, DATE(tweet.tweet_created_at), user.user_friends_count, COUNT(DISTINCT retweet.retweet_id) FROM retweet \
INNER JOIN tweet ON tweet.tweet_id = retweet.source_tweet_id \
INNER JOIN user ON user.user_id = tweet.user_id \
GROUP BY tweet.tweet_id \
ORDER BY COUNT(DISTINCT retweet.retweet_id) DESC', engine)
print(resRetweetUserFriends)

               tweet_id DATE(tweet.tweet_created_at)  user_friends_count  \
0   1053685028125884417                   2018-10-20                  82   
1   1053719282281078784                   2018-10-20                 398   
2   1053748203617607680                   2018-10-20                 728   
3   1053706908127780864                   2018-10-20                 728   
4   1053710745899167744                   2018-10-20                 210   
5   1053688093860073472                   2018-10-20                  82   
6   1053712251591348224                   2018-10-20                 144   
7   1053768308317659137                   2018-10-20                 144   
8   1053783521561432065                   2018-10-20                 325   
9   1053675634017239040                   2018-10-20                 728   
10  1053714186981715968                   2018-10-20                 144   
11  1053685031690997760                   2018-10-20                  82   
12  10536869

In [17]:
resAvgFriends = pd.read_sql_query('SELECT AVG(user_friends_count) FROM user', engine)
print(resAvgFriends)

   AVG(user_friends_count)
0              1939.257476


In [21]:
# Korelacja pomiędzy friends_count usera a łączną liczbą retweetów postów usera
# Wyłonienie najbardziej wpływowych użytkowników sieci
resRetweetUserFriends = pd.read_sql_query('SELECT user.user_id, user.user_friends_count, \
COUNT(DISTINCT retweet.retweet_id) AS \'Total retweets of users posts\', \
COUNT(DISTINCT tweet.tweet_id) AS \'Users distinct posts that got retweeted\' \
FROM retweet \
INNER JOIN tweet ON retweet.source_tweet_id = tweet.tweet_id \
INNER JOIN user ON tweet.user_id = user.user_id \
GROUP BY(user.user_id) \
ORDER BY COUNT(DISTINCT retweet.retweet_id) DESC', engine)
print(resRetweetUserFriends)

               user_id  user_friends_count  Total retweets of users posts  \
0   860301272376803332                  82                             18   
1           4037632102                 728                             15   
2  1000232495764426752                 398                             11   
3            141612654                 144                              7   
4             32739094                 210                              5   
5   849491378325401600                 325                              3   
6  1025796691461591041                 144                              3   
7            123283283                 180                              1   
8            881604271                4726                              1   

   Users distinct posts that got retweeted  
0                                        4  
1                                        3  
2                                        1  
3                                        4  
4   